In [1]:
import sys
import torchvision
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.9 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=8c77861feec7a23a3bf1dbe6c59b68859e4f5fb0ab5a7dc56df168796421561c
  Stored in directory: /root/.cache/pip/wheels/ff/55/fb/b27a466be754d2a06ffe0e37b248d844f090a63b51becea85d
Successfully built kaggle
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.4.5.1
    Uninstalling certifi-2020.4.5.1:
      Successfully uninstalled certifi-2020.4.5.1

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Dataset URL: https://www.kaggle.com/datasets/trolukovich/food5k-image-dataset
License(s): CC0-1.0
100%|████████████████████████████████████████| 426M/426M [

/root/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import foodgenius.model_builder as model_builder

effnetb2_5k_food_or_nonfood, effnetb2_transforms = model_builder.create_effnetb2_model(num_classes=2)

food_5k_transforms = torchvision.transforms.Compose([
    torchvision.transforms.TrivialAugmentWide(), effnetb2_transforms,
                                                    ])

food_5k_transforms

Compose(
    TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
)

In [ ]:
!pip install kaggle
!kaggle datasets download -d trolukovich/food5k-image-dataset

In [3]:
!unzip -o food5k-image-dataset.zip -d data/food5k

Archive:  food5k-image-dataset.zip
  inflating: data/food5k/evaluation/food/0.jpg  
  inflating: data/food5k/evaluation/food/1.jpg  
  inflating: data/food5k/evaluation/food/10.jpg  
  inflating: data/food5k/evaluation/food/100.jpg  
  inflating: data/food5k/evaluation/food/101.jpg  
  inflating: data/food5k/evaluation/food/102.jpg  
  inflating: data/food5k/evaluation/food/103.jpg  
  inflating: data/food5k/evaluation/food/104.jpg  
  inflating: data/food5k/evaluation/food/105.jpg  
  inflating: data/food5k/evaluation/food/106.jpg  
  inflating: data/food5k/evaluation/food/107.jpg  
  inflating: data/food5k/evaluation/food/108.jpg  
  inflating: data/food5k/evaluation/food/109.jpg  
  inflating: data/food5k/evaluation/food/11.jpg  
  inflating: data/food5k/evaluation/food/110.jpg  
  inflating: data/food5k/evaluation/food/111.jpg  
  inflating: data/food5k/evaluation/food/112.jpg  
  inflating: data/food5k/evaluation/food/113.jpg  
  inflating: data/food5k/evaluation/food/114.jpg  
  

In [11]:
from torchvision import datasets
import foodgenius.data_setup as data_setup

from pathlib import Path

data_dir = Path("data")

train_dir = data_dir / "food5k" / "training"
test_dir = data_dir / "food5k" / "validation"

train_dataloader_food5k, test_dataloader_food5k, class_names = data_setup.create_dataloaders(train_dir, test_dir, food_5k_transforms, 32, 2)

class_names 

['food', 'non_food']

In [12]:
import foodgenius.engine as engine

optimizer = torch.optim.Adam(effnetb2_5k_food_or_nonfood.parameters(), lr=0.001)

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)


effnetb2_5k_nonfood_results = engine.train(model=effnetb2_5k_food_or_nonfood.to('cuda:0'),
                                    train_dataloader=train_dataloader_food5k,
                                    test_dataloader=test_dataloader_food5k,
                                    optimizer=optimizer,
                                    loss_fn=loss_fn,
                                    epochs=10,
                                    device=torch.device('cuda'))


  0%|                                                                                      | 0/5 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
 20%|███████████████▌                                                              | 1/5 [00:21<01:25, 21.46s/it]

Epoch: 1 | train_loss: 0.3739 | train_acc: 0.9142 | test_loss: 0.2907 | test_acc: 0.9736


 40%|███████████████████████████████▏                                              | 2/5 [00:41<01:01, 20.50s/it]

Epoch: 2 | train_loss: 0.2943 | train_acc: 0.9582 | test_loss: 0.2801 | test_acc: 0.9648


 60%|██████████████████████████████████████████████▊                               | 3/5 [01:01<00:40, 20.47s/it]

Epoch: 3 | train_loss: 0.2854 | train_acc: 0.9640 | test_loss: 0.2727 | test_acc: 0.9785


 80%|██████████████████████████████████████████████████████████████▍               | 4/5 [01:22<00:20, 20.42s/it]

Epoch: 4 | train_loss: 0.2767 | train_acc: 0.9689 | test_loss: 0.2672 | test_acc: 0.9775


In [ ]:
import foodgenius.utils as utils

utils.save_model(model=effnetb2_5k_food_or_nonfood, target_dir="models", model_name='pretrained_effnetb2_feature_extractor_food_5k_food_or_nonfood.pth')

[INFO] Saving model to: models/09_pretrained_effnetb2_feature_extractor_food_101.pth
